![problem3](problem3.png)

In [7]:
from pyomo.environ import *

# Create a new instance of a ConcreteModel
model = ConcreteModel()

# Define a set representing the months
months = [1, 2, 3, 4]

# Define parameters (constants) for the model
prices = {'regular': 6, 'overtime': 7, 'third_party': 7.5}  # Prices for making per engine/K
demand = {1:6000, 2:5000, 3:7000, 4:5000}
initial_inventory =  1000 # Initial inventory at the beginning
storage_cost_per_ton = 1  # Cost to store each engine/K
warehouse_capacity = 500  # Minimum storage capacity of the warehouse
reg_per_month = 5000
ot_per_month = 500

# Decision Variables
model.reg = Var(months, domain=NonNegativeReals)
model.ot = Var(months, domain=NonNegativeReals)
model.third = Var(months, domain=NonNegativeReals)
model.Inventory = Var(months, domain=NonNegativeReals)

# Constraints
def inventory_balance_rule(model, t):
    if t == months[0]:
        return model.Inventory[t] == initial_inventory + model.reg[t] + model.ot[t] + model.third[t] - demand[t]
    return model.Inventory[t] == model.Inventory[t-1] + model.reg[t] + model.ot[t] + model.third[t] - demand[t]
model.InventoryBalance = Constraint(months, rule=inventory_balance_rule)

def warehouse_capacity_rule(model, t):
    return model.Inventory[t] >= warehouse_capacity
model.WarehouseCapacity = Constraint(months, rule=warehouse_capacity_rule)

def reg_rule(model,t):
    return model.reg[t] <= reg_per_month
model.reg_rule = Constraint(months, rule = reg_rule)

def ot_rule(model,t):
    return model.ot[t] <= ot_per_month
model.ot_rule = Constraint(months, rule = ot_rule)

def demand_rule(model,t):
    if t == months[0]:
        return initial_inventory+ model.reg[t] + model.ot[t] + model.third[t] >= demand[t]
    return model.Inventory[t-1]+ model.reg[t] + model.ot[t] + model.third[t] >= demand[t]
model.demand_rule = Constraint(months, rule = demand_rule)

model.C1 = Constraint(expr = sum(model.third[t] for t in months)<= 2000)


# Objective
def cost_rule(model):
    cost = sum(model.reg[t] * prices['regular']+ model.ot[t] * prices['overtime']+ model.third[t] * prices['third_party'] for t in months)
    storage_costs = sum((model.Inventory[t] * storage_cost_per_ton for t in months))
    return cost+ storage_costs
model.cost = Objective(rule=cost_rule, sense=minimize)

# Print the model
model.pprint()

9 Set Declarations
    InventoryBalance_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    Inventory_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    WarehouseCapacity_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    demand_rule_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    ot_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    ot_rule_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    reg_index : Siz

In [10]:
# Solving the model using the Gurobi solver
solver = SolverFactory('gurobi')
solver.solve(model, tee=True)

# Displaying the results for each month
for t in months:
    print('Month:', t)
    print(' Regular:', value(model.reg[t]))
    print(' Overtime:', value(model.ot[t]))
    print(' Third_party:', value(model.third[t]))
    print(' Inventory:', value(model.Inventory[t]))

print('\nCost:' ,model.cost(),'(1000s)')

Restricted license - for non-production use only - expires 2025-11-24
Read LP format model from file /var/folders/1k/03gv_xvd763c_7v1mkkyhljr0000gn/T/tmpyrp3fjjv.pyomo.lp
Reading time = 0.00 seconds
x1: 21 rows, 16 columns, 50 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.6.0 21G726)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 21 rows, 16 columns and 50 nonzeros
Model fingerprint: 0x2afa262b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 7e+03]
Presolve removed 14 rows and 4 columns
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6500000e+04   2.687500e+03   0.000000e+00      0s
       4    1.4025000e+05   0.000000e+00   0.000000e+00      0s

Solv